### Reto Precios

#### Analisis preliminar

En este ejercicio, se utilizarán modelos matemáticos, estadísticos y de aprendizaje automático para discernir cuáles son los factores que más afectan el precio de los departamentos en la Ciudad de México por metro cuadrado.

Para facilitar el procesamiento de datos, se realizó un análisis preliminar en Excel. Se proporcionó un conjunto de datos con 981 entradas, de las cuales 31 no se encuentran en la Ciudad de México, por lo que serán retiradas del análisis. Además, 942 entradas se concentran en la delegación Cuauhtémoc, por lo que las 8 entradas restantes serán eliminadas del análisis para evitar ruido innecesario.arDe las 942 entradas en la delegación Cuauhtémoc, un total de 33 no tienen especificada la colonia a la que pertenecen. De las 904 que sí especifican esta información, 902 se encuentran en la colonia Roma Norte o Roma Sur, con 2 excepciones en la colonia Condesa. Debido a estas discrepancias, el análisis que se presentará a continuación se realizará únicamente con las 907 entradas en el conjunto de datos para las cuales tenemos información suficiente para realizar un análisis estadístico en profundidad.n
Además, se eliminó información que no es relevante para el análisis. Esta información incluye el nombre del departamento, el subtítulo de la publicación, el enlace de la publicación, la fecha de publicación (ya que contamos con los días desde la publicación), la descripción extendida en la publicación (ya que toda la información presente en esta descripción se encuentra en otras métricas), y el ID (que se agregará automáticamente cuando la información sea convertida a un conjunto de datos, además de que hubo un desplazamiento debido a la limpieza inicial).

También se eliminaron las coordenadas del departamento (latitud y longitud), la divisa en la que está listado el precio (en todos los casos son pesos mexicanos), la medida de tiempo desde la publicación del departamento en el sitio del vendedor, el valor de la medida de tiempo asociada a la publicación (ya que existe una columna de días desde la publicación), la edad del departamento en años (todas las entradas son 0), el costo mensual (la mayoría no tiene un costo mensual asociado, casi todos los departamentos que cuentan con uno tienen un costo de 0 MXN, y el análisis de este trabajo se dirige a predecir el costo de los departamentos, no el costo de su renta asociada).

Además, se eliminaron las columnas de departamentos por piso, disposición, número de pisos, piso, orientación, y tipo de departamento, ya que la mayoría de las entradas están en blanco y no aportan información relevante para el análisis.
s A medida que avance el análisis, se utilizará la ingeniería de características en caso de ser necesario. Igualmente, se reevaluará el uso de las columnas restantes en caso de que se presente colinealidad, estas columnas sean perjudiciales para el análisis, o la información que proporcionan no sea útil para el análisis.

Las columnas que se utilizarán para el análisis son las siguientes: Colonia, Vendedor, Días desde la publicación, Amenidades, Baños, Sótanos, Lugares de estacionamiento, Número de recámaras, Metros cuadrados, Precio del departamento, y Precio por metro cuadrado. Las variables como vendedor y colonia son variables categóricas, por lo que se crearán sus respectivos dummies más adelante en el análisis seran creados mas adelante en el analisis.eran creados mas adelante en el analisis.


In [94]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import plotly.express as px
import xgboost as xgb
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [95]:
df = pd.read_csv('Datos_R_P.csv')
df = df.fillna(0)
print(df.head())

      Colonia                 Vendedorx  DDP  Amenidades  Baños  Sotanos  \
0    Roma Sur                 Investemx    5         3.0      2      0.0   
1  Roma Norte                   Avitana    7         2.0      1      0.0   
2    Roma Sur               Ebranuncios    4         0.0      3      2.0   
3    Roma Sur                      Houm  150         2.0      2      0.0   
4  Roma Norte  Contactobienesraicesgold   52         4.0      2      0.0   

   Lugares_Estacionamiento  Recamaras     m2  Precio_Final         PPMC  
0                        2          2  127.0       9000000  70866.14173  
1                        1          1   54.0       4264000  78962.96296  
2                        2          3  163.0       8093500  49653.37423  
3                        1          2   93.0       4000000  43010.75269  
4                        1          2  103.0       6732700  65366.01942  


In [127]:
for column in df.columns:
    fig = px.histogram(df, x=column)
    fig.show()

En lo anterior claramente se exhiben tres outlier, el primero: tres publicaciones que lleva más de 700 días, no consistente con el hecho que la edad del edificio es de 0 años, el segundo: un departamento con 23 baños, y el tercero: un departamento de 7000 metros cuadrados, eliminaremos estos datos y volveremos a graficar

In [52]:
##confirmamos los valores sin sentido 
print(df['Baños'].max())
print(df['DDP'].max())
print(df['m2'].max())

23
730
7210.0


In [53]:
##removemos las publicaciones con tiempo excesivo
dff = df[df['DDP'] < 365]
##buscamos id de outliers
maxid = {dff['Baños'].idxmax(),dff['m2'].idxmax()}
##los eliminamos
RP = dff.drop(maxid)

In [128]:
##graficamos con datos actualizados
for column in ['Baños','DDP','m2']:
    fig = px.histogram(RP, x=column)
    fig.show()

In [55]:
# Calculando la correlación entre las variables numéricas
numerical_cols = RP.select_dtypes(include=[np.number]).columns
correlation = RP[numerical_cols].corr()

In [56]:
price_sqm_corr = correlation['PPMC']
price_sqm_corr
## este calculo inicial de correlacion simplemente busca encontrar una tendencia lineal en algunos de los datos

DDP                       -0.152559
Amenidades                -0.098907
Baños                     -0.151969
Sotanos                   -0.014082
Lugares_Estacionamiento    0.016426
Recamaras                 -0.301102
m2                        -0.324175
Precio_Final               0.314527
PPMC                       1.000000
Name: PPMC, dtype: float64

In [57]:
for column in numerical_cols:
    if column != 'PPMC':
        fig = px.scatter(RP, x=column, y='PPMC')
        fig.show()

Fácilmente notamos que solo dos variables tienen correlación lineal débil con el precio por metro cuadrado, lo cual no es sorprendente pues son los metros cuadrados y el precio final del departamento, de cualquier modo, se hará una regresión lineal multi variada para luego compararla con el resto de los modelos.

In [131]:
RP_rlin = RP

In [132]:
RP_rlin = pd.get_dummies(RP_rlin, columns=['Colonia', 'Vendedorx'])
vendedor_dummies = [col for col in RP_rlin.columns if col.startswith('Vendedorx_')]

In [133]:
# Preparando los datos para la regresión lineal
# X contiene todas las columnas excepto 'PPMC', que es nuestra variable objetivo
X = RP_rlin.drop(['PPMC'], axis=1)
y = RP_rlin['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Creando y entrenando el modelo de regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Haciendo predicciones con el modelo de regresión lineal
y_pred = linear_model.predict(X_test)

In [134]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))


Mean Absolute Error: 5298.169280494971
Mean Squared Error: 54609337.58719564
Root Mean Squared Error: 7389.813095552257
R2: 0.7916335101906967


Parece que nuestro modelo lineal se ajusta mejor de lo que esperabamos, así pues, veamos que coeficientes son los que mas significancia tienen

In [135]:
# Añadiendo una constante a las variables independientes
X2 = sm.add_constant(X)

# Ajustando el modelo
model = sm.OLS(y.astype(float), X2.astype(float))
results = model.fit()

# Imprimiendo los coeficientes y los valores p
print(results.summary2())

                                   Results: Ordinary least squares
Model:                          OLS                         Adj. R-squared:                0.861     
Dependent Variable:             PPMC                        AIC:                           17327.2712
Date:                           2023-07-22 21:33            BIC:                           17928.7152
No. Observations:               842                         Log-Likelihood:                -8536.6   
Df Model:                       126                         F-statistic:                   42.48     
Df Residuals:                   715                         Prob (F-statistic):            4.74e-261 
R-squared:                      0.882                       Scale:                         4.4131e+07
-----------------------------------------------------------------------------------------------------
                                           Coef.     Std.Err.    t     P>|t|     [0.025      0.975]  
---------------

Los resultados son muy similares a verlo con vendedores, pero es importante notar que esto solamente es una regresion lineal, hay herramientas que pueden resultarnos más utiles para este analisis.

La variable objetivo es PPMC, y también estamos analizando el precio y los metros cuadrados, estas columnas introducen multicolinealidad al modelo, pues son variables redundantes, se harán análisis con y sin ellas más adelante

In [98]:
SC = [col for col in RP_rlin.columns if col not in vendedor_dummies]
RP_rlin2 = RP_rlin[SC]

In [99]:
# Preparando los datos para la regresión lineal
# X contiene todas las columnas excepto 'PPMC', que es nuestra variable objetivo
X = RP_rlin2.drop(['PPMC'], axis=1)
y = RP_rlin2['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Creando y entrenando el modelo de regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Haciendo predicciones con el modelo de regresión lineal
y_pred = linear_model.predict(X_test)

In [65]:
model = LinearRegression().fit(X, y)
# Añadiendo una constante a las variables independientes
X2 = sm.add_constant(X)

# Ajustando el modelo
model = sm.OLS(y.astype(float), X2.astype(float))
results = model.fit()

# Imprimiendo los coeficientes y los valores p
print(results.summary2())

                         Results: Ordinary least squares
Model:                   OLS                   Adj. R-squared:          0.832     
Dependent Variable:      PPMC                  AIC:                     17380.9008
Date:                    2023-07-22 13:08      BIC:                     17428.2586
No. Observations:        842                   Log-Likelihood:          -8680.5   
Df Model:                9                     F-statistic:             465.0     
Df Residuals:            832                   Prob (F-statistic):      1.63e-317 
R-squared:               0.834                 Scale:                   5.3368e+07
----------------------------------------------------------------------------------
                          Coef.     Std.Err.    t     P>|t|    [0.025     0.975]  
----------------------------------------------------------------------------------
const                   48544.0795  750.9072  64.6472 0.0000 47070.1844 50017.9747
DDP                        -7.

Los resultados son muy similares a verlo con vendedores, pero es importante notar que esto solamente es una regresión lineal, hay herramientas que pueden resultarnos más útiles para este análisis.

In [100]:
# Preparando los datos para la regresión lineal
# X contiene todas las columnas excepto 'PPMC', que es nuestra variable objetivo
X = RP_rlin2.drop(['PPMC','Precio_Final','m2'], axis=1)
y = RP_rlin2['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Creando y entrenando el modelo de regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Haciendo predicciones con el modelo de regresión lineal
y_pred = linear_model.predict(X_test)

In [101]:
model = LinearRegression().fit(X, y)
# Añadiendo una constante a las variables independientes
X2 = sm.add_constant(X)

# Ajustando el modelo
model = sm.OLS(y.astype(float), X2.astype(float))
results = model.fit()

# Imprimiendo los coeficientes y los valores p
print(results.summary2())

                          Results: Ordinary least squares
Model:                    OLS                   Adj. R-squared:          0.124     
Dependent Variable:       PPMC                  AIC:                     18770.8392
Date:                     2023-07-22 19:17      BIC:                     18808.7255
No. Observations:         842                   Log-Likelihood:          -9377.4   
Df Model:                 7                     F-statistic:             18.08     
Df Residuals:             834                   Prob (F-statistic):      1.83e-22  
R-squared:                0.132                 Scale:                   2.7875e+08
-----------------------------------------------------------------------------------
                           Coef.     Std.Err.    t    P>|t|     [0.025     0.975]  
-----------------------------------------------------------------------------------
const                    49807.3357 1679.4389 29.6571 0.0000  46510.9120 53103.7594
DDP               

Sigue exisitinedo mucha colinealidad, esto tambien puede deberse al usod e variables dicretas como lo son el numero de baños, esperemos los proximos modelos nos den un mejor insight en que afecta al PPMC

#### ARBOLES DE DECISION

In [66]:
AD = RP
AD = pd.get_dummies(AD, columns=['Colonia', 'Vendedorx'])
X = AD.drop(['PPMC'], axis=1)
y = AD['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTreeRegressor()

# Entrenar el modelo
tree.fit(X_train, y_train)

# Predecir
y_pred_tree = tree.predict(X_test)

# Calcular métricas
mae_tree = metrics.mean_absolute_error(y_test, y_pred_tree)
mse_tree = metrics.mean_squared_error(y_test, y_pred_tree)
rmse_tree = np.sqrt(mse_tree)
r2_tree = metrics.r2_score(y_test, y_pred_tree)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_tree)
print('MSE:', mse_tree)
print('RMSE:', rmse_tree)
print('R^2:', r2_tree)

# Obtener la importancia de las características
importances_tree = tree.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_tree = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_tree
})

# Ordenar el DataFrame por importancia
importance_df_tree = importance_df_tree.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_tree)

Decision Tree Metrics:
MAE: 2545.535886106509
MSE: 41768068.90199769
RMSE: 6462.822054025447
R^2: 0.8549506635232988
Feature Importances:
                                Feature  Importance
7                          Precio_Final    0.561429
6                                    m2    0.403761
0                                   DDP    0.021051
21         Vendedorx_Buscatuhogarmexico    0.003437
11  Vendedorx_Aire Y Acero Inmobiliaria    0.002604
..                                  ...         ...
90                  Vendedorx_Mexicosil    0.000000
55           Vendedorx_Enders Inmuebles    0.000000
92     Vendedorx_Morakalli Inmobiliaria    0.000000
52     Vendedorx_Distritocapitalcondesa    0.000000
53                Vendedorx_Ebranuncios    0.000000

[128 rows x 2 columns]


#### AHORA HAREMOS LO MISMO SIN LXS VENDEDORXS 

In [137]:
SC = [col for col in AD.columns if col not in vendedor_dummies]
AD2 = AD[SC]

X = AD2.drop(['PPMC'], axis=1)
y = AD2['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTreeRegressor()

# Entrenar el modelo
tree.fit(X_train, y_train)

# Predecir
y_pred_tree = tree.predict(X_test)

# Calcular métricas
mae_tree = metrics.mean_absolute_error(y_test, y_pred_tree)
mse_tree = metrics.mean_squared_error(y_test, y_pred_tree)
rmse_tree = np.sqrt(mse_tree)
r2_tree = metrics.r2_score(y_test, y_pred_tree)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_tree)
print('MSE:', mse_tree)
print('RMSE:', rmse_tree)
print('R^2:', r2_tree)

# Obtener la importancia de las características
importances_tree = tree.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_tree = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_tree
})

# Ordenar el DataFrame por importancia
importance_df_tree = importance_df_tree.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_tree)

Decision Tree Metrics:
MAE: 2528.629476047338
MSE: 26517044.264193088
RMSE: 5149.470289669908
R^2: 0.9266346245804373
Feature Importances:
                   Feature  Importance
7             Precio_Final    0.593690
6                       m2    0.399748
1               Amenidades    0.002574
0                      DDP    0.001790
2                    Baños    0.001148
5                Recamaras    0.000343
9         Colonia_Roma Sur    0.000232
8       Colonia_Roma Norte    0.000229
4  Lugares_Estacionamiento    0.000224
3                  Sotanos    0.000023


In [105]:
X = AD2.drop(['PPMC','Precio_Final','m2'], axis=1)
y = AD2['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTreeRegressor()

# Entrenar el modelo
tree.fit(X_train, y_train)

# Predecir
y_pred_tree = tree.predict(X_test)

# Calcular métricas
mae_tree = metrics.mean_absolute_error(y_test, y_pred_tree)
mse_tree = metrics.mean_squared_error(y_test, y_pred_tree)
rmse_tree = np.sqrt(mse_tree)
r2_tree = metrics.r2_score(y_test, y_pred_tree)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_tree)
print('MSE:', mse_tree)
print('RMSE:', rmse_tree)
print('R^2:', r2_tree)

# Obtener la importancia de las características
importances_tree = tree.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_tree = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_tree
})

# Ordenar el DataFrame por importancia
importance_df_tree = importance_df_tree.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_tree)

Decision Tree Metrics:
MAE: 9535.80554392005
MSE: 234040057.79484323
RMSE: 15298.367814732499
R^2: 0.1973083350716487
Feature Importances:
                   Feature  Importance
0                      DDP    0.340662
1               Amenidades    0.200792
5                Recamaras    0.164490
4  Lugares_Estacionamiento    0.130155
2                    Baños    0.083141
7         Colonia_Roma Sur    0.048116
6       Colonia_Roma Norte    0.017594
3                  Sotanos    0.015050


Se obtienen mejores resultados con Bosques de decisión consistentemente removiendo lxs vendedorxs, y no es difícil notar que sin el precio por metro cuadrado empieza a haber características que resaltan, como los días desde la publicación, las amenidades y recamaras.

De cualquier manera, continuemos, hay dos modelos con los que personalmente quiero analizar este set de datos.


#### RANDOM FORESTS

In [68]:
RF = RP

RF = pd.get_dummies(RF, columns=['Colonia', 'Vendedorx'])

X = RF.drop(['PPMC'], axis=1)
y = RF['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

forest = RandomForestRegressor(random_state=0)

# Entrenar el modelo
forest.fit(X_train, y_train)

# Predecir
y_pred_for = forest.predict(X_test)

# Calcular métricas
mae_for = metrics.mean_absolute_error(y_test, y_pred_for)
mse_for = metrics.mean_squared_error(y_test, y_pred_for)
rmse_for = np.sqrt(mse_for)
r2_for = metrics.r2_score(y_test, y_pred_for)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_for)
print('MSE:', mse_for)
print('RMSE:', rmse_for)
print('R^2:', r2_for)

# Obtener la importancia de las características
importances_for = forest.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_for = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_for
})

# Ordenar el DataFrame por importancia
importance_df_for = importance_df_for.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_for)

Decision Tree Metrics:
MAE: 1612.883562530526
MSE: 11079519.289545815
RMSE: 3328.5911869056276
R^2: 0.9691021711225669
Feature Importances:
                             Feature  Importance
7                       Precio_Final    0.570508
6                                 m2    0.389991
21      Vendedorx_Buscatuhogarmexico    0.005877
4            Lugares_Estacionamiento    0.005292
0                                DDP    0.005156
..                               ...         ...
71                Vendedorx_Hipoo Mx    0.000000
16                Vendedorx_Balinter    0.000000
111         Vendedorx_Remax Satelite    0.000000
49   Vendedorx_Cravioto Inmobiliaria    0.000000
33        Vendedorx_Century21 Urbana    0.000000

[128 rows x 2 columns]


Estos resultados son mejores incluso que los arboles de decision sin vendedorxs, así pues, busquemos si es posible mejorarlo quitando el ruido

In [69]:
SC = [col for col in RF.columns if col not in vendedor_dummies]
RF2 = RF[SC]

X = RF2.drop(['PPMC'], axis=1)
y = RF2['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

forest = RandomForestRegressor()

# Entrenar el modelo
forest.fit(X_train, y_train)

# Predecir
y_pred_for = forest.predict(X_test)

# Calcular métricas
mae_for = metrics.mean_absolute_error(y_test, y_pred_for)
mse_for = metrics.mean_squared_error(y_test, y_pred_for)
rmse_for = np.sqrt(mse_for)
r2_for = metrics.r2_score(y_test, y_pred_for)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_for)
print('MSE:', mse_for)
print('RMSE:', rmse_for)
print('R^2:', r2_for)

# Obtener la importancia de las características
importances_for = forest.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_for = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_for
})

# Ordenar el DataFrame por importancia
importance_df_for = importance_df_for.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_for)

Decision Tree Metrics:
MAE: 1418.7105914395293
MSE: 7584600.481445142
RMSE: 2754.0153379102926
R^2: 0.9755814778512474
Feature Importances:
                   Feature  Importance
7             Precio_Final    0.576898
6                       m2    0.396011
0                      DDP    0.009488
5                Recamaras    0.004748
4  Lugares_Estacionamiento    0.004554
1               Amenidades    0.004331
2                    Baños    0.001815
8       Colonia_Roma Norte    0.000863
9         Colonia_Roma Sur    0.000813
3                  Sotanos    0.000479


In [109]:
X = RF2.drop(['PPMC','m2','Precio_Final'], axis=1)
y = RF2['PPMC']

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

forest = RandomForestRegressor()

# Entrenar el modelo
forest.fit(X_train, y_train)

# Predecir
y_pred_for = forest.predict(X_test)

# Calcular métricas
mae_for = metrics.mean_absolute_error(y_test, y_pred_for)
mse_for = metrics.mean_squared_error(y_test, y_pred_for)
rmse_for = np.sqrt(mse_for)
r2_for = metrics.r2_score(y_test, y_pred_for)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_for)
print('MSE:', mse_for)
print('RMSE:', rmse_for)
print('R^2:', r2_for)

# Obtener la importancia de las características
importances_for = forest.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_for = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_for
})

# Ordenar el DataFrame por importancia
importance_df_for = importance_df_for.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_for)

Decision Tree Metrics:
MAE: 8422.092790465485
MSE: 153588906.49082103
RMSE: 12393.099147946046
R^2: 0.34648515206139874
Feature Importances:
                   Feature  Importance
0                      DDP    0.382640
5                Recamaras    0.178542
1               Amenidades    0.176796
4  Lugares_Estacionamiento    0.119742
2                    Baños    0.060774
7         Colonia_Roma Sur    0.031798
6       Colonia_Roma Norte    0.031352
3                  Sotanos    0.018356


 Hasta ahora solo hemos percibido mejoras en el modelado, y la reduccion de ruido en el modelo causada por lxs vendedorxs tmabien nos ha dado mucha más informacion, veamos si un approach con un modelo distinto nos da resultados similares.

### XGBOOST

Utilizo XGBoost, pues en sí mismo es un algoritmo robusto que puede manejar una variedad de tipos de datos y distribuciones, y a menudo funciona bien incluso sin un preprocesamiento extenso de los datos, esto es, podemos evadir el uso de técnicas de oversmapling o clustering simplemente por lo robusto y eficiente que es este modelo

In [110]:
XG = RP
XG = pd.get_dummies(XG, columns=['Colonia', 'Vendedorx'])
X = XG.drop(['PPMC'], axis=1)
y = XG['PPMC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Crear el modelo
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Entrenar el modelo
xg_reg.fit(X_train, y_train)
# Predecir
y_pred_xgb = xg_reg.predict(X_test)

# Calcular métricas
mae_xgb = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = metrics.r2_score(y_test, y_pred_xgb)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_xgb)
print('MSE:', mse_xgb)
print('RMSE:', rmse_xgb)
print('R^2:', r2_xgb)

# Obtener la importancia de las características
importances_xgb = xg_reg.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_xgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_xgb
})

# Ordenar el DataFrame por importancia
importance_df_xgb = importance_df_xgb.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_xgb)


Decision Tree Metrics:
MAE: 1808.6888797692307
MSE: 7992408.680874555
RMSE: 2827.0848379336894
R^2: 0.9672396916234998
Feature Importances:
                              Feature  Importance
7                        Precio_Final    0.401918
6                                  m2    0.278998
113       Vendedorx_Rent A House Cdmx    0.083622
5                           Recamaras    0.049329
1                          Amenidades    0.027488
..                                ...         ...
57   Vendedorx_Espacios Inmobiliarios    0.000000
97                 Vendedorx_Nouhouse    0.000000
70           Vendedorx_Highend Mexico    0.000000
99        Vendedorx_Oscarjuarezmontao    0.000000
56         Vendedorx_Espacio Inmobili    0.000000

[128 rows x 2 columns]


In [111]:
SC = [col for col in XG.columns if col not in vendedor_dummies]
XG2 = XG[SC]

X = XG2.drop(['PPMC',], axis=1)
y = XG2['PPMC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Crear el modelo
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Entrenar el modelo
xg_reg.fit(X_train, y_train)
# Predecir
y_pred_xgb = xg_reg.predict(X_test)

# Calcular métricas
mae_xgb = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = metrics.r2_score(y_test, y_pred_xgb)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_xgb)
print('MSE:', mse_xgb)
print('RMSE:', rmse_xgb)
print('R^2:', r2_xgb)

# Obtener la importancia de las características
importances_xgb = xg_reg.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_xgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_xgb
})

# Ordenar el DataFrame por importancia
importance_df_xgb = importance_df_xgb.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_xgb)


Decision Tree Metrics:
MAE: 1725.3993004448964
MSE: 9465559.348406736
RMSE: 3076.6149171462353
R^2: 0.9737643683405609
Feature Importances:
                   Feature  Importance
7             Precio_Final    0.439747
6                       m2    0.363927
5                Recamaras    0.088240
1               Amenidades    0.073488
4  Lugares_Estacionamiento    0.013811
3                  Sotanos    0.009088
0                      DDP    0.005122
2                    Baños    0.004644
8       Colonia_Roma Norte    0.001933
9         Colonia_Roma Sur    0.000000


Queda claro que hay dos columnas que ofuscan severamente nuestro análisis y estas siendo metros cuadrados y precio por metro cuadrado, pues ambas están ligadas directamente a la variable objetivo del análisis. 

Con XGBOOST especificamente realizaremo un analisis de la importancia de las variables sin m2 y precio final, sin m2 unicamente y sin precio final unicamente. para determinar que afecta más los precios de departamentos por m2

In [138]:
XG = RP
XG = pd.get_dummies(XG, columns=['Colonia', 'Vendedorx'])
SC = [col for col in XG.columns if col not in vendedor_dummies]
XG2 = XG[SC]

X = XG2.drop(['PPMC','Precio_Final','m2'], axis=1)
y = XG2['PPMC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Crear el modelo
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Entrenar el modelo
xg_reg.fit(X_train, y_train)
# Predecir
y_pred_xgb = xg_reg.predict(X_test)

# Calcular métricas
mae_xgb = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = metrics.r2_score(y_test, y_pred_xgb)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_xgb)
print('MSE:', mse_xgb)
print('RMSE:', rmse_xgb)
print('R^2:', r2_xgb)

# Obtener la importancia de las características
importances_xgb = xg_reg.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_xgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_xgb
})

# Ordenar el DataFrame por importancia
importance_df_xgb = importance_df_xgb.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_xgb)

Decision Tree Metrics:
MAE: 11183.868889444158
MSE: 232580555.39083984
RMSE: 15250.591968538134
R^2: 0.4377195987479675
Feature Importances:
                   Feature  Importance
5                Recamaras    0.301251
4  Lugares_Estacionamiento    0.178294
1               Amenidades    0.138110
0                      DDP    0.112737
2                    Baños    0.103411
6       Colonia_Roma Norte    0.088405
3                  Sotanos    0.077793
7         Colonia_Roma Sur    0.000000


##### Importancia mayor a 0.1
- Recamaras
- Lugares de estacionamiento
- Amenidades
- DDP

#### Sin metros cuadrados

In [142]:
XG = RP
XG = pd.get_dummies(XG, columns=['Colonia', 'Vendedorx'])
SC = [col for col in XG.columns if col not in vendedor_dummies]
XG2 = XG[SC]

X = XG2.drop(['PPMC','m2'], axis=1)
y = XG2['PPMC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Crear el modelo
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Entrenar el modelo
xg_reg.fit(X_train, y_train)
# Predecir
y_pred_xgb = xg_reg.predict(X_test)

# Calcular métricas
mae_xgb = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = metrics.r2_score(y_test, y_pred_xgb)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_xgb)
print('MSE:', mse_xgb)
print('RMSE:', rmse_xgb)
print('R^2:', r2_xgb)

# Obtener la importancia de las características
importances_xgb = xg_reg.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_xgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_xgb
})

# Ordenar el DataFrame por importancia
importance_df_xgb = importance_df_xgb.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_xgb)

Decision Tree Metrics:
MAE: 6958.924931775889
MSE: 110680122.5797633
RMSE: 10520.462089650022
R^2: 0.6823321999163994
Feature Importances:
                   Feature  Importance
5                Recamaras    0.338388
6             Precio_Final    0.183598
1               Amenidades    0.121000
2                    Baños    0.105809
4  Lugares_Estacionamiento    0.068575
0                      DDP    0.067955
7       Colonia_Roma Norte    0.061574
3                  Sotanos    0.053103
8         Colonia_Roma Sur    0.000000


##### Importancia mayor a 0.1
- Recamaras
- Precio final **
- Amenidades


#### Sin precio final

In [141]:
XG = RP
XG = pd.get_dummies(XG, columns=['Colonia', 'Vendedorx'])
SC = [col for col in XG.columns if col not in vendedor_dummies]
XG2 = XG[SC]

X = XG2.drop(['PPMC','Precio_Final'], axis=1)
y = XG2['PPMC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Crear el modelo
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Entrenar el modelo
xg_reg.fit(X_train, y_train)
# Predecir
y_pred_xgb = xg_reg.predict(X_test)

# Calcular métricas
mae_xgb = metrics.mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = metrics.mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = metrics.r2_score(y_test, y_pred_xgb)

# Imprimir métricas
print('Decision Tree Metrics:')
print('MAE:', mae_xgb)
print('MSE:', mse_xgb)
print('RMSE:', rmse_xgb)
print('R^2:', r2_xgb)

# Obtener la importancia de las características
importances_xgb = xg_reg.feature_importances_

# Crear un DataFrame para visualizar los resultados
importance_df_xgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances_xgb
})

# Ordenar el DataFrame por importancia
importance_df_xgb = importance_df_xgb.sort_values(by='Importance', ascending=False)

print('Feature Importances:')
print(importance_df_xgb)

Decision Tree Metrics:
MAE: 8060.105270931212
MSE: 139701138.9477371
RMSE: 11819.523634552159
R^2: 0.5406993864288354
Feature Importances:
                   Feature  Importance
5                Recamaras    0.209039
4  Lugares_Estacionamiento    0.203862
6                       m2    0.138520
7       Colonia_Roma Norte    0.116690
1               Amenidades    0.109494
0                      DDP    0.109367
2                    Baños    0.082652
3                  Sotanos    0.030377
8         Colonia_Roma Sur    0.000000


##### Importancia mayor a 0.1
- Lugares de estacionamiento
- Recamaras
- Amenidades
-  m2 **
- Roma Norte